In [ ]:

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler, PolynomialFeatures
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils.class_weight import compute_class_weight
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import (
    classification_report, accuracy_score, roc_auc_score, roc_curve,
    precision_recall_curve, confusion_matrix
)

from imblearn.under_sampling import RandomUnderSampler

import shap
import xgboost as xgb
from catboost import CatBoostClassifier, Pool

RNG = 42
np.random.seed(RNG)

# ----------------- PATHS -----------------
DATA_PATH = r"E:\PXA252_BH\OlderFiles20250512\class_all_with_chronic_names.csv"

# 1) LOAD + BASIC CLEAN
df = pd.read_csv(DATA_PATH)

# Harmonize binary target (1->0, 2/3->1) to handle both your binary and ternary cases
df = df[df['class'].isin([1, 2, 3])]
df['class'] = df['class'].replace({1: 0, 2: 1, 3: 1})
y = df['class']

drop_cols = ['HASHED_PERSONID', 'ENCNTR_ID_SI', 'DIAG_DT_TM', 'ICD', 'DIAGNOSIS_DISPLAY', 'DIAG_TYPE']
df = df.drop(columns=[c for c in drop_cols if c in df.columns], errors='ignore')
X_full = df.drop(columns=['class'])

# Identify categorical columns on the raw data
cat_cols_raw = X_full.select_dtypes(include=['object', 'category']).columns.tolist()

# Replace infs with NaN uniformly
X_full = X_full.replace([np.inf, -np.inf], np.nan)

# 2) PARALLEL DATA VIEWS
#    - X_cb: for CatBoost (categoricals as string, numerics imputed only)
#    - X_num: for XGB & LogReg (categoricals label-encoded to ints, numerics imputed)
# ---- X_cb for CatBoost
X_cb = X_full.copy()

# Ensure categorical columns are strings with an explicit Missing token
for col in cat_cols_raw:
    X_cb[col] = X_cb[col].astype(str)
    X_cb[col] = X_cb[col].replace({'nan': 'Missing'}).fillna('Missing')

# Impute numeric columns (leaving categoricals as strings)
num_cols_cb = X_cb.select_dtypes(include=[np.number]).columns
imp_num_cb = SimpleImputer(strategy='mean')
X_cb[num_cols_cb] = imp_num_cb.fit_transform(X_cb[num_cols_cb])

# ---- X_num for XGB / LogReg
X_num = X_full.copy()
# Label-encode categoricals to integers
label_encoders = {}
for col in cat_cols_raw:
    le = LabelEncoder()
    X_num[col] = le.fit_transform(X_num[col].astype(str))  # encodes Missing as a distinct int
    label_encoders[col] = le

# Impute all numeric columns
num_cols_num = X_num.select_dtypes(include=[np.number]).columns
imp_num_num = SimpleImputer(strategy='mean')
X_num[num_cols_num] = imp_num_num.fit_transform(X_num[num_cols_num])

# 3) CONSISTENT FEATURE SELECTION (RF-based, on X_num)
#    - Select top N features from X_num (int-coded cats + numerics)
#    - Apply the same feature subset to X_cb (by column names)
N_TOP = 30
rf_fs = RandomForestClassifier(n_estimators=100, random_state=RNG)
rf_fs.fit(X_num, y)
top_features = pd.Series(rf_fs.feature_importances_, index=X_num.columns).nlargest(N_TOP).index.tolist()

X_num = X_num[top_features].copy()
X_cb  = X_cb[[c for c in top_features if c in X_cb.columns]].copy()  # some rare columns may be absent

# Update cat_cols according to the selected features
cat_cols_cb = [c for c in cat_cols_raw if c in X_cb.columns]

# 4) SPLITS (same indices across both views for fair comparison)
X_train_val_idx, X_test_idx, y_train_val, y_test = train_test_split(
    np.arange(len(y)), y, stratify=y, test_size=0.2, random_state=RNG
)
X_train_idx, X_val_idx, y_train, y_val = train_test_split(
    X_train_val_idx, y_train_val, stratify=y_train_val, test_size=0.25, random_state=RNG
)

def take_rows(Xdf, idx):
    return Xdf.iloc[idx].reset_index(drop=True)

# Views for CatBoost
X_cb_train = take_rows(X_cb, X_train_idx)
X_cb_val   = take_rows(X_cb, X_val_idx)
X_cb_test  = take_rows(X_cb, X_test_idx)

# Views for XGB/LogReg
X_num_train = take_rows(X_num, X_train_idx)
X_num_val   = take_rows(X_num, X_val_idx)
X_num_test  = take_rows(X_num, X_test_idx)

# Targets
y_train = pd.Series(y_train).reset_index(drop=True)
y_val   = pd.Series(y_val).reset_index(drop=True)
y_test  = pd.Series(y_test).reset_index(drop=True)

# 5) MODEL 1: CATBOOST (with undersampling, strict dtype handling) + SHAP
# Undersample training to balance
rus = RandomUnderSampler(sampling_strategy=1.0, random_state=RNG)
X_cb_train_bal, y_train_bal = rus.fit_resample(X_cb_train, y_train)

# Make sure cats stay strings everywhere (re-assert; harmless if already)
for col in cat_cols_cb:
    for frame in (X_cb_train_bal, X_cb_train, X_cb_val, X_cb_test):
        frame[col] = frame[col].astype(str).fillna('Missing')

# Cat feature indices relative to X_cb_train_bal columns
cat_features_idx = [X_cb_train_bal.columns.get_loc(c) for c in cat_cols_cb if c in X_cb_train_bal.columns]

# Train CatBoost
cb_model = CatBoostClassifier(
    iterations=500,
    learning_rate=0.03,
    depth=5,
    l2_leaf_reg=8,
    eval_metric='F1',
    random_seed=RNG,
    cat_features=cat_features_idx,
    early_stopping_rounds=50,
    class_weights={0: 1.1, 1: 1},
    verbose=100
)
cb_model.fit(X_cb_train_bal, y_train_bal, eval_set=(X_cb_val, y_val), use_best_model=True)

def evaluate_catboost(mdl, Xd, yd, label):
    pool = Pool(Xd, label=yd, cat_features=cat_features_idx)
    y_pred = mdl.predict(pool)
    y_prob = mdl.predict_proba(pool)[:, 1]
    print(f"\n📊 {label} (CatBoost) Report:")
    print(classification_report(yd, y_pred))
    print(f"✅ Acc: {accuracy_score(yd, y_pred):.4f}")
    print(f"🎯 ROC-AUC: {roc_auc_score(yd, y_prob):.4f}")
    return yd, y_prob

ytr_cb, ytrp_cb = evaluate_catboost(cb_model, X_cb_train, y_train, "Train")
yva_cb,  yvap_cb = evaluate_catboost(cb_model, X_cb_val,   y_val,   "Validation")
yte_cb,  ytep_cb = evaluate_catboost(cb_model, X_cb_test,  y_test,  "Test")

# Threshold optimization (maximize recall for class 0 with precision > 0.5)
prec, rec, thr = precision_recall_curve((yva_cb == 0).astype(int), 1 - yvap_cb)
best_thresh, max_recall = 0.5, 0
for p, r, t in zip(prec, rec, np.append(thr, 1.0)):
    if p > 0.5 and r > max_recall:
        max_recall = r
        best_thresh = 1 - t
optimal_thresh_cb = best_thresh
print(f"🔧 CatBoost class-0 recall-optimized threshold = {optimal_thresh_cb:.3f}")


# 6) MODEL 2: XGBOOST (with calibration) + SHAP
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
sample_weights = np.where(y_train == 0, class_weights[0], class_weights[1])
num_class0 = int(np.sum(y_train == 0))
num_class1 = int(np.sum(y_train == 1))
scale_pos_weight = num_class0 / max(1, num_class1)

xgb_model = xgb.XGBClassifier(
    n_estimators=200,
    learning_rate=0.03,
    max_depth=5,
    subsample=0.6,
    colsample_bytree=0.6,
    reg_lambda=10,
    reg_alpha=5,
    scale_pos_weight=scale_pos_weight,
    objective='binary:logistic',
    eval_metric='auc',
    random_state=RNG
)
xgb_model.fit(X_num_train, y_train, sample_weight=sample_weights)

# Calibrate (sigmoid)
xgb_cal = CalibratedClassifierCV(xgb_model, method='sigmoid', cv='prefit')
xgb_cal.fit(X_num_val, y_val)

def evaluate_prob_model(model, Xd, yd, label, threshold=0.636):
    y_prob = model.predict_proba(Xd)[:, 1]
    y_pred = (y_prob >= threshold).astype(int)
    print(f"\n📊 {label} (XGB Calibrated) Report (thr={threshold:.3f}):")
    print(classification_report(yd, y_pred))
    print(f"✅ Acc: {accuracy_score(yd, y_pred):.4f}")
    print(f"🎯 ROC-AUC: {roc_auc_score(yd, y_prob):.4f}")
    return yd, y_prob, y_pred

THRESH_XGB = 0.636
ytr_xgb, ytrp_xgb, _ = evaluate_prob_model(xgb_cal, X_num_train, y_train, "Train", THRESH_XGB)
yva_xgb, yvap_xgb, _ = evaluate_prob_model(xgb_cal, X_num_val,   y_val,   "Validation", THRESH_XGB)
yte_xgb, ytep_xgb, ytepred_xgb = evaluate_prob_model(xgb_cal, X_num_test,  y_test,  "Test", THRESH_XGB)

# SHAP for XGB (use pre-calibration model)
os.makedirs("shap_outputs/xgb", exist_ok=True)
xgb_explainer = shap.TreeExplainer(xgb_model)
xgb_shap_test = xgb_explainer.shap_values(X_num_test)
xgb_base_value = xgb_explainer.expected_value
xgb_feature_names = list(X_num_train.columns)

plt.figure(figsize=(10, 6))
shap.summary_plot(xgb_shap_test, features=X_num_test, feature_names=xgb_feature_names, show=False)
plt.tight_layout(); plt.savefig("shap_outputs/xgb/summary_beeswarm.png", dpi=200); plt.close()

plt.figure(figsize=(10, 6))
shap.summary_plot(xgb_shap_test, features=X_num_test, feature_names=xgb_feature_names, plot_type="bar", show=False)
plt.tight_layout(); plt.savefig("shap_outputs/xgb/summary_bar.png", dpi=200); plt.close()

i = 0
plt.figure(figsize=(9, 6))
shap.plots._waterfall.waterfall_legacy(xgb_base_value, xgb_shap_test[i], feature_names=xgb_feature_names, max_display=20, show=False)
plt.tight_layout(); plt.savefig(f"shap_outputs/xgb/waterfall_idx{i}.png", dpi=200); plt.close()

print("✅ XGBoost SHAP plots saved to shap_outputs/xgb")

# 7) MODEL 3: LOGISTIC REGRESSION (Elastic-Net on poly+SelectKBest) + SHAP
# Standardize numeric-coded view before poly
scaler = StandardScaler()
X_num_train_s = scaler.fit_transform(X_num_train)
X_num_val_s   = scaler.transform(X_num_val)
X_num_test_s  = scaler.transform(X_num_test)

# Polynomial interactions (degree=2, interactions only)
poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
X_tr_poly = poly.fit_transform(X_num_train_s)
X_va_poly = poly.transform(X_num_val_s)
X_te_poly = poly.transform(X_num_test_s)
poly_feature_names = poly.get_feature_names_out(X_num_train.columns)

# Feature selection (top K)
K = 100 if X_tr_poly.shape[1] >= 100 else X_tr_poly.shape[1]
selector = SelectKBest(score_func=f_classif, k=K)
X_tr_sel = selector.fit_transform(X_tr_poly, y_train)
X_va_sel = selector.transform(X_va_poly)
X_te_sel = selector.transform(X_te_poly)
selected_feature_names = np.array(poly_feature_names)[selector.get_support()]

# Grid search LR (Elastic-Net)
param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10],
    'penalty': ['elasticnet'],
    'l1_ratio': [0.1, 0.5, 0.9],
    'solver': ['saga'],
    'class_weight': [None, 'balanced'],
    'max_iter': [5000]
}
grid_lr = GridSearchCV(
    LogisticRegression(),
    param_grid,
    scoring='roc_auc',
    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=RNG),
    n_jobs=-1,
    verbose=1
)
grid_lr.fit(np.vstack([X_tr_sel, X_va_sel]), pd.concat([y_train, y_val]).values)
best_lr = grid_lr.best_estimator_
print("🔍 LR best params:", grid_lr.best_params_)

def evaluate_lr(model, Xd, yd, label):
    y_pred = model.predict(Xd)
    y_prob = model.predict_proba(Xd)[:, 1]
    print(f"\n📊 {label} (LogReg) Report:")
    print(classification_report(yd, y_pred))
    print(f"✅ Acc: {accuracy_score(yd, y_pred):.4f}")
    print(f"🎯 ROC-AUC: {roc_auc_score(yd, y_prob):.4f}")
    return yd, y_prob

ytr_lr, ytrp_lr = evaluate_lr(best_lr, X_tr_sel, y_train, "Train")
yva_lr, yvap_lr = evaluate_lr(best_lr, X_va_sel, y_val,   "Validation")
yte_lr, ytep_lr = evaluate_lr(best_lr, X_te_sel, y_test,  "Test")

import shap
import matplotlib.pyplot as plt
import os
from catboost import Pool

import shap, matplotlib.pyplot as plt, os
from catboost import Pool

os.makedirs("shap_outputs/catboost", exist_ok=True)

cb_shap_all = cb_model.get_feature_importance(
    Pool(X_cb_test, label=y_test, cat_features=cat_features_idx),
    type="ShapValues"
)

cb_shap_values = cb_shap_all[:, :-1]   # per-feature SHAP values
cb_expected_value = cb_shap_all[:, -1] # expected value for each sample

feature_names = list(X_cb_test.columns)

# --- Beeswarm ---
plt.figure(figsize=(10, 6))
shap.summary_plot(cb_shap_values, features=X_cb_test, feature_names=feature_names, show=False)
plt.tight_layout()
plt.savefig("shap_outputs/catboost/summary_beeswarm.png", dpi=300)
plt.close()

# --- Bar plot ---
plt.figure(figsize=(10, 6))
shap.summary_plot(cb_shap_values, features=X_cb_test, feature_names=feature_names, plot_type="bar", show=False)
plt.tight_layout()
plt.savefig("shap_outputs/catboost/summary_bar.png", dpi=300)
plt.close()

# --- Local waterfall for one test example ---
i = 0
plt.figure(figsize=(9, 6))
shap.plots._waterfall.waterfall_legacy(cb_expected_value[i], cb_shap_values[i],
                                       feature_names=feature_names, max_display=20, show=False)
plt.tight_layout()
plt.savefig(f"shap_outputs/catboost/waterfall_idx{i}.png", dpi=300)
plt.close()

print("✅ CatBoost SHAP plots generated using native get_feature_importance(type='ShapValues').")


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.preprocessing import StandardScaler
import shap
import matplotlib.pyplot as plt
import pickle

# Step 1: Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

# Step 2: Standardize
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Step 3: Train Logistic Regression
log_reg = LogisticRegression(max_iter=1000, class_weight='balanced', random_state=42)
log_reg.fit(X_train_scaled, y_train)

# Step 4: Evaluation
y_pred = log_reg.predict(X_test_scaled)
print("✅ Logistic Regression Classification Report:\n")
print(classification_report(y_test, y_pred))
print(f"🎯 ROC-AUC: {roc_auc_score(y_test, log_reg.predict_proba(X_test_scaled)[:,1]):.4f}")

# Step 5: SHAP Explanation
shap.initjs()
lr_explainer = shap.LinearExplainer(log_reg, X_train_scaled, feature_perturbation="interventional")
lr_shap_values = lr_explainer.shap_values(X_test_scaled)

# Optional: Save SHAP values
with open("shap_values_log_reg.pkl", "wb") as f:
    pickle.dump(lr_shap_values, f)

# Optional: Save explainer
with open("shap_explainer_log_reg.pkl", "wb") as f:
    pickle.dump(lr_explainer, f)

# Step 6: SHAP Summary Plot
shap.summary_plot(lr_shap_values, X_test, feature_names=X.columns)
